# 03. summarising the orthogrouping - stats - this might turn into an r- notebook

In [ ]:
mamba activate dl20-orthogrouping
conda info --envs

In [ ]:
dl20=(
at6923 at6929 at7143
at8285 at9104 at9336 at9503
at9578 at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

In [ ]:
# move just NLR (orthofinder) orthogroups into a folder

mkdir -p tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/

for orthogroup in $(cat tmp/just_manually_annotated_final_actual/nlr_containing_orthogroups.txt)
do 
cp tmp/just_manually_annotated_final_actual/manually_curated_orthogroups/${orthogroup}.fa tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/
done

In [ ]:
# align these using mafft

bash src/mafft_for_03.sh

#Contents of bash script is:
##!/bin/bash
#set -xe

#for orthogroup in $(cat tmp/just_manually_annotated_final_actual/nlr_containing_orthogroups.txt)
#do 
#mafft --auto tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/${orthogroup}.fa > tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/${orthogroup}_mafft.fa
#done

In [ ]:
# run amas

python3 ./src/AMAS.py summary -f fasta -d aa -i tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/*_mafft.fa

In [ ]:
mv summary.txt tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/

In [ ]:
#make fasta files of nlrs from orthogroups purely for assigning pseudogenes to orthogroups.

python3 ./src/fasta_formatter_general_with_an_orthofinder_list_1.2.py -t tmp/just_manually_annotated_final_actual/nlr_sequences_orthogroup_map_just_for_assigning_nt_pseudogenes.txt output/per_accession_nt_cds/*.fasta

In [ ]:
cat tmp/just_manually_annotated_final_actual/just_nlrs_from_nlr_containing_orthogroups_cds_nt/*.fa \
    > tmp/just_manually_annotated_final_actual/all_nlrs_so_far_assigned_orthogroups_cds_nt.fasta

In [ ]:
## All-by-all diamond to produce broad orthogroups

In [ ]:
cat tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/*.fa > \
    tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps.fasta

In [ ]:
diamond makedb --in tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps.fasta \
    -d tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps
# running a search in blastp mode
diamond blastp -d tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps \
   -q tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps.fasta \
   -o tmp/just_manually_annotated_final_actual/all_sequences_from_nlr_containing_orthogroups_aa_cds_reps_matches.tsv

In [ ]:
### Manual check of orthogroups which had matches of 85% sequence identity 

In [ ]:
# Produce a list of the sequences in the intermediate broad orthogroups and check that they are all unique

In [ ]:
python src/produce_list_of_fasta_ids.py tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/*.fa > intermediate_broad_orthogroups.txt

In [ ]:
head intermediate_broad_orthogroups.txt

In [ ]:
# They are all unique

In [ ]:
# Then merge the sequences into the broad groups - put in table and produce fasta files

In [ ]:
python3 src/produce_list_of_orthogroups_to_merge.py \
    -t tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/intermediate_orthogroups_to_be_merged_to_broad_orthogroups.txt


In [ ]:
# Then - do clustering on the broad groups to produce the allelic orthogroups

In [ ]:
# produce fasta files for these

In [ ]:
cd tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups

In [ ]:
bash merge_to_broad_orthogroups.sh


In [ ]:
bash remove_files_that_were_merged.sh

In [ ]:
# Then alignments, stats, trees

In [ ]:
# get the sequence to broad orthogroup mapping

In [ ]:
python src/produce_list_of_fasta_ids.py tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups/*.fa \
    > broad_orthogroups.txt

In [ ]:
cd tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups

In [ ]:
ls *.fa > samples.txt

In [ ]:
sed -i 's/\.fa//g' samples.txt

In [ ]:
bash ../../../../../src/mafft.sh

In [ ]:
# work out which sequences are in which orthogroup

In [ ]:
pwd

In [ ]:
cat input/neigbourhoods_new2/*.bed > input/neigbourhoods_new2/all_neighbour_hoods.bed

In [ ]:
bedtools intersect -a tmp/just_manually_annotated_final_actual/all_final_nlrs.bed \
   -b input/neigbourhoods_new2/all_neighbour_hoods.bed -wb > tmp/just_manually_annotated_final_actual/all_nlr_neighbourhoods.txt

In [ ]:
head tmp/just_manually_annotated_final_actual/all_nlr_neighbourhoods.txt

## Running cd-hit for splitting test

In [ ]:
head broad_orthogroups_that_need_splitting.txt

In [ ]:
for orthogroup in $(cat tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups/samples.txt)
do 
    cd-hit -i tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups/${orthogroup}.fa \
    -o tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups/cd_hit_20231026/${orthogroup}_90.fa \
    -c 0.90
done

In [ ]:
bash src/mafft.sh

In [ ]:
python p-distance_script_v3.py -t prot -p average -s ~ -f *_mafft.fa > average_distance_table.txt

In [ ]:
cd tmp/just_manually_annotated_final_actual/just_nlr_containing_orthogroups/broad_orthogroups/actual_broad_orthogroups/cd_hit_20231026

In [ ]:
python3 ../../../../../../src/cd-hit_output_parser.py *_85.fa.clstr > cd_hit_summary_test_85.txt